In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')

### TASK: Heart Disease Prediction
* STEP 1: Data Preprocessing
* STEP 2: Model Application
* STEP 3: Model Selection

##### Data Preprocessing
* Dataset: 2020 annual CDC survey data of 400k adults related to their health status[319795*18]
    * prediction (y): HeartDisease (YES or NO) [heavily unbalanced]
    * features (X): 17 variables (13 categories and 4 numericals)

In [2]:
df = pd.read_csv('heart_2020.csv')
display(df)

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319790,Yes,27.41,Yes,No,No,7.0,0.0,Yes,Male,60-64,Hispanic,Yes,No,Fair,6.0,Yes,No,No
319791,No,29.84,Yes,No,No,0.0,0.0,No,Male,35-39,Hispanic,No,Yes,Very good,5.0,Yes,No,No
319792,No,24.24,No,No,No,0.0,0.0,No,Female,45-49,Hispanic,No,Yes,Good,6.0,No,No,No
319793,No,32.81,No,No,No,0.0,0.0,No,Female,25-29,Hispanic,No,No,Good,12.0,No,No,No


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319795 entries, 0 to 319794
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   HeartDisease      319795 non-null  object 
 1   BMI               319795 non-null  float64
 2   Smoking           319795 non-null  object 
 3   AlcoholDrinking   319795 non-null  object 
 4   Stroke            319795 non-null  object 
 5   PhysicalHealth    319795 non-null  float64
 6   MentalHealth      319795 non-null  float64
 7   DiffWalking       319795 non-null  object 
 8   Sex               319795 non-null  object 
 9   AgeCategory       319795 non-null  object 
 10  Race              319795 non-null  object 
 11  Diabetic          319795 non-null  object 
 12  PhysicalActivity  319795 non-null  object 
 13  GenHealth         319795 non-null  object 
 14  SleepTime         319795 non-null  float64
 15  Asthma            319795 non-null  object 
 16  KidneyDisease     31

In [4]:
cat_cols = ['Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex', 'AgeCategory', 
             'Race', 'Diabetic', 'PhysicalActivity', 'GenHealth', 'Asthma', 'KidneyDisease', 'SkinCancer']
num_cols = ['BMI','PhysicalHealth','MentalHealth','SleepTime']

# rescale the data
df1 = df[num_cols]
df1 = MinMaxScaler().fit_transform(df1)
df1 = pd.DataFrame(df1, columns=['BMI','PhysicalHealth','MentalHealth','SleepTime'])

# binarize the data
df2 = df[cat_cols]
cat_cols_encoded = []
for col in cat_cols:
    cat_cols_encoded += [f"{col}_{cat}" for cat in list(df2[col].unique())]
    
oh_encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_cols = oh_encoder.fit_transform(df2)
df2 = pd.DataFrame(encoded_cols, columns=cat_cols_encoded)

# standardize the data
X = pd.concat([df1, df2], axis=1)
y = pd.DataFrame(df['HeartDisease'].apply(lambda x:0 if x=='No' else 1))
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# before training
X_train, X_test, y_train, y_test = train_test_split(X_scaled, np.array(y).reshape(-1), test_size=0.1)

# calculate test error
def err(preds,lbl):
    n = len(lbl)
    return sum((lbl-preds)**2) / n

##### Model Application

In [5]:
# Logistic Regression
clf1 = LogisticRegression().fit(X_train, y_train)
score1 = clf1.score(X_train, y_train)
y_test_pred1 = clf1.predict(X_test)
print('LR -- Score: {}, Test Error: {}'.format(score1,err(y_test_pred1,y_test)))

# Random Forest
clf2 = RandomForestClassifier().fit(X_train, y_train)
score2 = clf2.score(X_train, y_train)
y_test_pred2 = clf2.predict(X_test)
print('RF -- Score: {}, Test Error: {}'.format(score2,err(y_test_pred2,y_test)))

LR -- Score: 0.9158139777287494, Test Error: 0.083270794246404
RF -- Score: 0.9964908013828327, Test Error: 0.09699812382739212


In [8]:
# Artificial Neural Network
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(1,input_shape=(num_input,) ,activation='sigmoid'))
model.compile(optimizer='sgd', metrics=['accuracy'], loss='binary_crossentropy')
history1 = model.fit(X_train, y_train, epochs=100,validation_split=0.1)
_, model_accuracy_train = model.evaluate(X_train,y_train)
y_test_pred3 = model.predict(X_test).reshape(-1)
print('NN -- Score: {}, Test Error: {}'.format(model_accuracy_train,err(y_test_pred3,y_test)))

Epoch 1/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2557 - accuracy: 0.9064 - val_loss: 0.2296 - val_accuracy: 0.9153
Epoch 2/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2274 - accuracy: 0.9156 - val_loss: 0.2290 - val_accuracy: 0.9150
Epoch 3/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2271 - accuracy: 0.9156 - val_loss: 0.2288 - val_accuracy: 0.9153
Epoch 4/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2270 - accuracy: 0.9155 - val_loss: 0.2292 - val_accuracy: 0.9144
Epoch 5/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2271 - accuracy: 0.9159 - val_loss: 0.2288 - val_accuracy: 0.9152
Epoch 6/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2271 - accuracy: 0.9157 - val_loss: 0.2290 - val_accuracy: 0.9149
Epoch 7/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2271 - accuracy: 0.9158 - val_loss: 0.2289 -

Epoch 57/100
8095/8095 [==============================] - 9s 1ms/step - loss: 0.2271 - accuracy: 0.9158 - val_loss: 0.2288 - val_accuracy: 0.9157
Epoch 58/100
8095/8095 [==============================] - 9s 1ms/step - loss: 0.2270 - accuracy: 0.9157 - val_loss: 0.2288 - val_accuracy: 0.9153
Epoch 59/100
8095/8095 [==============================] - 9s 1ms/step - loss: 0.2271 - accuracy: 0.9155 - val_loss: 0.2289 - val_accuracy: 0.9147
Epoch 60/100
8095/8095 [==============================] - 9s 1ms/step - loss: 0.2271 - accuracy: 0.9158 - val_loss: 0.2291 - val_accuracy: 0.9149
Epoch 61/100
8095/8095 [==============================] - 9s 1ms/step - loss: 0.2271 - accuracy: 0.9157 - val_loss: 0.2288 - val_accuracy: 0.9152
Epoch 62/100
8095/8095 [==============================] - 9s 1ms/step - loss: 0.2271 - accuracy: 0.9157 - val_loss: 0.2288 - val_accuracy: 0.9153
Epoch 63/100
8095/8095 [==============================] - 9s 1ms/step - loss: 0.2270 - accuracy: 0.9156 - val_loss: 0.2294 -

##### Model Selection

In [7]:
# Logistic Regression
params_lr = {'C': np.logspace(-3,3,7), 'penalty': ['l1','l2']}
lr = LogisticRegression()
clf1 = GridSearchCV(lr, params_lr, cv=3).fit(X_train, y_train)
y_test_pred1 = clf1.predict(X_test)
print('After GridSearchCV LR with best parameters {}: \n    Score: {}, Test Error: {}'.format(clf1.best_estimator_, clf1.best_score_, err(y_test_pred1,y_test)))

# Random Forest
params_rf = {'n_estimators':[20, 40, 60, 80, 100], 'max_depth': [2,3,4]}
rf = RandomForestClassifier()
clf2 = GridSearchCV(rf, params_rf, cv=3).fit(X_train, y_train)
y_test_pred2 = clf2.predict(X_test)
print('After GridSearchCV RF with best parameters {}: \n    Score: {}, Test Error: {}'.format(clf2.best_estimator_, clf2.best_score_, err(y_test_pred2,y_test)))

# # Support Vector Machine
# params_svm = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
# svc = SVC()
# clf3 = GridSearchCV(svc, params_svm, cv=3).fit(X_train, y_train)
# y_test_pred3 = clf3.predict(X_test)
# print('After GridSearchCV SVM with best parameters {}: \n    Score: {}, Test Error: {}'.format(clf3.best_estimator_, clf3.best_score_,err(y_test_pred3,y_test)))

After GridSearchCV LR with best parameters LogisticRegression(C=0.01): 
    Score: 0.9158139798754953, Test Error: 0.08323952470293934
After GridSearchCV RF with best parameters RandomForestClassifier(max_depth=4, n_estimators=40): 
    Score: 0.9146187656934103, Test Error: 0.0841776110068793


In [9]:
# Artificial Neural Network
num_input = X_train.shape[1]
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(15,input_shape=(num_input,) ,activation='tanh'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='sgd', metrics=['accuracy'], loss='binary_crossentropy')
history2 = model.fit(X_train, y_train, epochs=100,validation_split=0.1)
_, model_accuracy_train = model.evaluate(X_train,y_train)
y_test_pred3 = model.predict(X_test).reshape(-1)
print('NN -- Score: {}, Test Error: {}'.format(model_accuracy_train,err(y_test_pred3,y_test)))

Epoch 1/100
8095/8095 [==============================] - 13s 2ms/step - loss: 0.2480 - accuracy: 0.9088 - val_loss: 0.2308 - val_accuracy: 0.9138
Epoch 2/100
8095/8095 [==============================] - 12s 1ms/step - loss: 0.2279 - accuracy: 0.9155 - val_loss: 0.2295 - val_accuracy: 0.9144
Epoch 3/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2269 - accuracy: 0.9156 - val_loss: 0.2289 - val_accuracy: 0.9141
Epoch 4/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2264 - accuracy: 0.9158 - val_loss: 0.2284 - val_accuracy: 0.9142
Epoch 5/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2260 - accuracy: 0.9159 - val_loss: 0.2282 - val_accuracy: 0.9141
Epoch 6/100
8095/8095 [==============================] - 12s 1ms/step - loss: 0.2257 - accuracy: 0.9158 - val_loss: 0.2281 - val_accuracy: 0.9141
Epoch 7/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2256 - accuracy: 0.9159 - val_loss: 0.2278 -

Epoch 57/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2235 - accuracy: 0.9165 - val_loss: 0.2276 - val_accuracy: 0.9141
Epoch 58/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2234 - accuracy: 0.9165 - val_loss: 0.2274 - val_accuracy: 0.9141
Epoch 59/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2234 - accuracy: 0.9165 - val_loss: 0.2276 - val_accuracy: 0.9144
Epoch 60/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2234 - accuracy: 0.9164 - val_loss: 0.2276 - val_accuracy: 0.9139
Epoch 61/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2234 - accuracy: 0.9164 - val_loss: 0.2276 - val_accuracy: 0.9141
Epoch 62/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2234 - accuracy: 0.9163 - val_loss: 0.2276 - val_accuracy: 0.9143
Epoch 63/100
8095/8095 [==============================] - 10s 1ms/step - loss: 0.2234 - accuracy: 0.9164 - val_loss: 0

In [6]:
# Artificial Neural Network
num_input = X_train.shape[1]
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(25,input_shape=(num_input,) ,activation='tanh'))
model.add(tf.keras.layers.Dense(15,activation='tanh'))
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))
model.compile(optimizer='sgd', metrics=['accuracy'], loss='binary_crossentropy')
history = model.fit(X_train, y_train, epochs=100,validation_split=0.1)
_, model_accuracy_train = model.evaluate(X_train,y_train)
y_test_pred3 = model.predict(X_test).reshape(-1)
print('NN -- Score: {}, Test Error: {}'.format(model_accuracy_train,err(y_test_pred3,y_test)))

Epoch 1/100
8095/8095 [==============================] - 12s 1ms/step - loss: 0.2422 - accuracy: 0.9110 - val_loss: 0.2311 - val_accuracy: 0.9128
Epoch 2/100
8095/8095 [==============================] - 12s 2ms/step - loss: 0.2278 - accuracy: 0.9152 - val_loss: 0.2295 - val_accuracy: 0.9132
Epoch 3/100
8095/8095 [==============================] - 13s 2ms/step - loss: 0.2269 - accuracy: 0.9154 - val_loss: 0.2289 - val_accuracy: 0.9142
Epoch 4/100
8095/8095 [==============================] - 12s 1ms/step - loss: 0.2263 - accuracy: 0.9158 - val_loss: 0.2284 - val_accuracy: 0.9134
Epoch 5/100
8095/8095 [==============================] - 12s 1ms/step - loss: 0.2259 - accuracy: 0.9156 - val_loss: 0.2281 - val_accuracy: 0.9136
Epoch 6/100
8095/8095 [==============================] - 13s 2ms/step - loss: 0.2256 - accuracy: 0.9159 - val_loss: 0.2283 - val_accuracy: 0.9135
Epoch 7/100
8095/8095 [==============================] - 14s 2ms/step - loss: 0.2253 - accuracy: 0.9161 - val_loss: 0.2284 -

Epoch 57/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2225 - accuracy: 0.9166 - val_loss: 0.2284 - val_accuracy: 0.9146
Epoch 58/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2224 - accuracy: 0.9166 - val_loss: 0.2285 - val_accuracy: 0.9141
Epoch 59/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2225 - accuracy: 0.9167 - val_loss: 0.2283 - val_accuracy: 0.9143
Epoch 60/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2224 - accuracy: 0.9165 - val_loss: 0.2285 - val_accuracy: 0.9147
Epoch 61/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2223 - accuracy: 0.9166 - val_loss: 0.2283 - val_accuracy: 0.9145
Epoch 62/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2224 - accuracy: 0.9166 - val_loss: 0.2284 - val_accuracy: 0.9148
Epoch 63/100
8095/8095 [==============================] - 11s 1ms/step - loss: 0.2223 - accuracy: 0.9168 - val_loss: 0